In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_excel("Training.xlsx")
df2 = pd.read_excel("Validation.xlsx")

In [ ]:
df1.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df2.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [ ]:
df1.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df1.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)

df2.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df2.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [ ]:
df_train = pd.concat([
    df1[['Emotion', 'sentence']],
    df1[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df1[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = pd.concat([
    df2[['Emotion', 'sentence']],
    df2[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df2[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)


In [ ]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

In [ ]:
df_train.to_csv("training_data.csv", index=False)
df_valid.to_csv("validation_data.csv", index=False)

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("training_data.csv")
df_valid = pd.read_csv("validation_data.csv")

In [ ]:
from konlpy.tag import Okt

In [ ]:
train = pd.read_csv("training_data.csv")

In [ ]:
okt = Okt()

temp = []
for i in train['sentence']:
    temp.append(okt.morphs(i))
    
temp = pd.Series(temp)
train['token'] = temp

In [ ]:
train.info()

In [ ]:
train.to_csv("training_data-token.csv", index=False)

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("training_data-token.csv")

In [ ]:
df_train['token']

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(df_train['token'], vector_size=1000, window=5, min_count=1, workers=4, sg=1)

In [ ]:
model.wv.vectors.shape

In [ ]:
import joblib

joblib.dump(model, 'word_model.joblib')

In [5]:
import pandas as pd

# 데이터를 불러옵니다.
data = pd.read_csv('training_data-token.csv')  # 데이터 파일 경로에 맞게 수정하세요.

# 각각의 감정에 대한 비율을 설정합니다.
desired_ratio = {
    '불안': 5000,
    '분노': 5000,
    '상처': 5000,
    '슬픔': 5000,
    '당황': 5000,
    '기쁨': 5000
}
# 선택한 데이터를 저장할 빈 데이터프레임을 생성합니다.
selected_data = pd.DataFrame(columns=data.columns)

# 각각의 감정에 대해 적절한 비율로 데이터를 선택합니다.
for emotion, ratio in desired_ratio.items():
    emotion_data = data[data['Emotion'] == emotion].sample(n=ratio, random_state=42)  # 적절한 비율로 샘플링
    selected_data = selected_data.append(emotion_data)

# 선택된 데이터를 확인합니다.
print(selected_data)

# 선택된 데이터를 새로운 파일로 저장합니다.
selected_data.to_csv('selected_data.csv', index=False)  # 저장할 파일명에 맞게 수정하세요.


AttributeError: 'DataFrame' object has no attribute 'append'

In [3]:
import pandas as pd

# 데이터를 불러옵니다.
data = pd.read_csv('training_data-token.csv')
emotion_counts = data['Emotion'].value_counts()
print(emotion_counts)

Emotion
불안    26301
분노    26000
상처    25957
슬픔    25814
당황    24936
기쁨    16947
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
from sklearn.svm import SVC
from joblib import load, dump

model = load("word_model.joblib")

X = [get_sent_embedding(model, 1000, tokens) for tokens in df_train['token']]
y = df_train['Emotion']

# SVM 분류 모델 학습
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)

# 모델 저장
dump(svm_model, 'svm_model.joblib')


In [ ]:
svm_model = 1
joblib.dump(svm_model, 'svm_model.joblib')